<a href="https://colab.research.google.com/github/yuvaajithansightspectrum/CPI_Automation/blob/main/CPI_Collab_Script_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Connecting To Drive

from google.colab import drive

# Check if Google Drive is already mounted
if not hasattr(drive, 'mounted') or not drive.mount('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive is already mounted.")

#----------------------------------------------

#----------PIP Install Req File-----------------
!pip install -r '/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/Req_Files/requirements.txt'
!pip install rpy2==3.5.16

#----------------------------------------

import pandas as pd
import os
#import shutil
from datetime import datetime
#Est Imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math
from openpyxl import load_workbook
import warnings
#Imports to run R within Python
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri


warnings.filterwarnings("ignore", category=FutureWarning)

# OUTPUT File Path
Output_Path=r'/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/OutPut_File/ARCH_TimeSeries1.xlsx'
#Output_Path1=r'C:\Users\SSMEPZ\3D Objects\CP\CPI\2023\test.xlsx'
#LOG File Path
Log_DF = pd.DataFrame(columns=['LOGS','TIME'])
current_date = datetime.now().strftime('%Y-%m-%d')

Log_path=r'/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/LOG\log_'+current_date+'.txt'

#=======Estimation File Path ============================================================

UMQ_path =  r"/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/Estimation_Input_Files/bdd_UQ.xlsx"
UMQ_Sheet_Name="UQ"

AJ_path = r"/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/Estimation_Input_Files/bdd_AJ.xlsx"
AJ_Sheet_Name="AJ"

SH_path =  r"/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/Estimation_Input_Files/bdd_SH.xlsx"
SH_Sheet_Name="SH"

#======================Default file path to store predictions temp ===============================
Temp_pred_path = r"/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/Estimation_Input_Files/Temp_Pred_File.xlsx"


#============================ Input File Path ===========================================
# Emirates codes for input FUJ, RAK, DXB, ABD.

# =============== Input processing MONTH (3 digit) & YEAR(2 didgit)  Ex: Month: JAN , Year=24.(If you are going to process January 2024 data)
param_path = r'/content/sample_data/PARAM.xlsx'

dtype_dict = {
    'CODE_INDX': 'str',
    'CPI_INDX': 'str',
    'START_ROW': 'int64',
    'END_ROW': 'int64',
    'MONTH': 'str',
    'YEAR': 'str'
}
try:
    param_df = pd.read_excel(param_path,dtype=dtype_dict) #dtype=dtype_dict
except FileNotFoundError:
    print(f"Error: File not found at path '{param_path}'")
except Exception as e:
    print(f"An error occurred: {e}")


def extract_row_variables(df, emirate, col):
    retuen_val = df.loc[df['EMIRATE'] == emirate, col].values
    if len(retuen_val) == 0:
        return []
    value = retuen_val[0]

    if isinstance(value, str) and ',' in value or col=='CODE_INDX' or col=='CPI_INDX':
        return [int(x.strip()) for x in value.split(',')]
    elif col=='START_ROW' or col=='END_ROW':
        return int(value)
    else:
        return str(value)


prc_month=extract_row_variables(param_df,'FUJ','MONTH')
prc_year=extract_row_variables(param_df,'FUJ','YEAR')


#=========================================================================================


#--FUJ
FUJ_path = r'/content/sample_data/FUJ.xlsx'    # Path of the input file
FUJ_code_col_to_read = extract_row_variables(param_df,'FUJ','CODE_INDX')  # Ex: if CODE exists in A cell means its index is '0', if 'B' means 1 and so on. (Ex value '0' if CODE in A cell)
FUJ_cpi_col_to_read = extract_row_variables(param_df,'FUJ','CPI_INDX') # Index of the Excel cell where CPI values exist Ex: if CPI value in C column of Excel means the value will be '2'.
FUJ_start_row=extract_row_variables(param_df,'FUJ','START_ROW') # Row number appears in Excel where CPI first value starts (Ex: if CPI value of CODE 00 start at row number 7 menas we have to give it as input)
FUJ_end_row=extract_row_variables(param_df,'FUJ','END_ROW') #  Row number appears in Excel where CPI last value ends (Ex: if last CPI is at row number 174th row menas we have to give it as input 174)
FUJ_cpi_column_name = [prc_month+'_'+prc_year+'_FUJ']

#--RAK
RAK_path = r'/content/sample_data/RAK.xlsx'
RAK_code_col_to_read = extract_row_variables(param_df,'RAK','CODE_INDX') # Ex: if CODE exists in A cell means its index is '0', if 'B' means 1 and so on. (Ex value '0' if CODE in A cell)
RAK_cpi_col_to_read = extract_row_variables(param_df,'RAK','CPI_INDX') # Index of the Excel cell where CPI values exist Ex: if CPI value in C column of Excel means the value will be '2'.
RAK_start_row=extract_row_variables(param_df,'RAK','START_ROW') # Row number appears in Excel where CPI first value starts (Ex: if CPI value of CODE 00 start at row number 7 menas we have to give it as input)
RAK_end_row=extract_row_variables(param_df,'RAK','END_ROW') #  Row number appears in Excel where CPI last value ends (Ex: if last CPI is at row number 174th row menas we have to give it as input 174)
RAK_cpi_column_name = [prc_month+'_'+prc_year+'_RAK']


#--ABD
ABD_path = r'/content/sample_data/ABD.xlsx'
ABD_code_col_to_read = extract_row_variables(param_df,'ABD','CODE_INDX') # Ex: if CODE exists in A cell means its index is '0', if 'B' means 1 and so on. (Ex value '0' if CODE in A cell)
ABD_cpi_col_to_read = extract_row_variables(param_df,'ABD','CPI_INDX') # Index of the Excel cell where CPI values exist Ex: if CPI value in C column of Excel means the value will be '2'.
ABD_start_row=extract_row_variables(param_df,'ABD','START_ROW') # Row number appears in Excel where CPI first value starts (Ex: if CPI value of CODE 00 start at row number 7 menas we have to give it as input)
ABD_end_row=extract_row_variables(param_df,'ABD','END_ROW') #  Row number appears in Excel where CPI last value ends (Ex: if last CPI is at row number 174th row menas we have to give it as input 174)
ABD_cpi_column_name = [prc_month+'_'+prc_year+'_ABU']

#--DXB
DXB_path = r'/content/sample_data/DUB.xlsx'
DXB_code_col_to_read = extract_row_variables(param_df,'DUB','CODE_INDX') # Ex: if CODE exists in combined cells like A,B and C means. We need to give its all index value as input.
DXB_cpi_col_to_read =extract_row_variables(param_df,'DUB','CPI_INDX') # Index of the Excel cell where CPI values exist Ex: if CPI value in C column of Excel means the value will be '2'.
DXB_start_row=extract_row_variables(param_df,'DUB','START_ROW')   # Row number appears in Excel where CPI first value starts (Ex: if CPI value of CODE 00 start at row number 7 menas we have to give it as input)
DXB_end_row=extract_row_variables(param_df,'DUB','END_ROW') #  Row number appears in Excel where CPI last value ends (Ex: if last CPI is at row number 174th row menas we have to give it as input 174)
DXB_cpi_column_name = [prc_month+'_'+prc_year+'_DUB']


#========================= Flags Def ===========================

coicop_flg=0
emi_weight_flg=0
fuj_rd_flg=0
rak_rd_flg=0
abd_rd_flg=0
dub_rd_flg=0
estimation_flg=0
arch_flg=0

print('Readed')
#==============================COICOP FILE READ START (Default File Path)==================================================================
COICOP_path = r'/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/Std_COICOP_Code/COICOP_Lvls.xlsx'
columns_to_read = [0,1,2]
new_column_names = ['CODE','DESC_AR','DESC_EN']
start_row=0
end_row=170
try:
    if not os.path.exists(COICOP_path):
        raise FileNotFoundError(f"The file '{COICOP_path}' does not exist.")
    else:
        COICOP_DF= pd.read_excel(COICOP_path, usecols=columns_to_read, skiprows=start_row, nrows=(end_row - start_row),dtype=str, engine='openpyxl' )
        COICOP_DF.columns = new_column_names
        for col in new_column_names:
            COICOP_DF[col] = COICOP_DF[col].astype(str)
        #print(COICOP_DF.dtypes)
        #print(COICOP_DF,'COIP')
        COICOP_DF_rows = COICOP_DF.shape[0]
        Log_DF.loc[len(Log_DF)] = ['COICOP File:'+str(COICOP_DF_rows)+' Readed at', current_date]
        coicop_flg=1
except FileNotFoundError as e:
    Log_DF.loc[len(Log_DF)] = [e, current_date]
    coicop_flg=0
#==================================== Weightage File Read (Default File Path) =============================

#for 4 Emirates Providing data
Weightage_path = r'/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/Std_CPI_weightage/CPI_WEIGHTAGES.xlsx'

#For 3 Emirates which we will calc CPI using Estimation
Weight_path=r'/content/drive/MyDrive/CPI_Automation/CPI_Input_Files/Std_EST_Weightage/Est_Weight_Uq_Aj_Sh.xlsx'

try:
    if not os.path.exists(Weightage_path):
        raise FileNotFoundError(f"The file '{FUJ_path}' does not exist.")
    else:
        Emirate_Weightage = pd.read_excel(Weightage_path, dtype={'CODE':str}, engine='openpyxl')
        #columns_to_convert = ['ABU_DHABI','DUBAI','SHARJAH','AJMAN','UMM_AL_QUWAIN','RAS_AL_KHAIMAH','FUJAIRAH','STATE_WEIGHT_BY_STATE_TO_HOUSEHOLDS','COUNTRY_WEIGHT_BY_COUNTRY_FOR_EMIRATI_FAMILI']
        #columns_to_convert = ['ABU_WEIGHT','DUB_WEIGHT','SHJ_WEIGHT','AJM_WEIGHT','UQM_WEIGHT','RAK_WEIGHT','FUJ_WEIGHT','GLOBALE','ALL_EMI_MUL_WEIGHT']
        #for column in columns_to_convert:
            #Emirate_Weightage[column] = pd.to_numeric(Emirate_Weightage[column], errors='coerce')
        Emirate_Weightage_rows = Emirate_Weightage.shape[0]
        Log_DF.loc[len(Log_DF)] = ['Emirates Weightages File:'+str(Emirate_Weightage_rows)+' Rows Readed at', current_date]
        emi_weight_flg=1
except FileNotFoundError as e:
    Log_DF.loc[len(Log_DF)] = [e, current_date]
    emi_weight_flg=0
#===============================================================================================================
#==================================  READ INPUT FILES ===================================================


def Read_Excel(path,col_index,cpi_index,start_row,end_row,cpi_column_name,datatype):
    read_Excel_df = pd.read_excel(path,dtype=datatype, skiprows=start_row-2, nrows=(end_row - start_row)+1,engine='openpyxl' ) #dtype=str
    #print(read_Excel_df)
    #with pd.ExcelWriter(path, engine='openpyxl', mode='a') as writer:
        #read_Excel_df.to_excel(writer, sheet_name='NewSheetName3', index=False)
    concat_code_col = read_Excel_df.iloc[:, col_index]
    concat_cpi_col = read_Excel_df.iloc[:, cpi_index]
    concat_code_col = concat_code_col.fillna('')
    read_Excel_df['CODE'] = concat_code_col.astype(str).agg(''.join, axis=1)
    read_Excel_df[cpi_column_name] = concat_cpi_col
    v_read_Excel_df=read_Excel_df[['CODE']+cpi_column_name]
    v_read_Excel_df.at[0, 'CODE'] = '00'
    v_read_Excel_df = v_read_Excel_df.assign(CODE=v_read_Excel_df['CODE'].str.replace(r'[_\*\.\-\ ]', '', regex=True))
    v_read_Excel_df[v_read_Excel_df['CODE'].str.len() <= 4]
    def def_level(codes):
        prefix = ""
        result = []
        for code in codes:
            if len(code) == 2 and code in ['01','02','03','04','05','06','07','08','09','10','11','12','13']:
                prefix = f"{code}"
                result.append(code)
            else:
                result.append(f"{prefix}")
        return result
    def add_zero(code, other_value):
        if other_value in ['01','02','03','04','05','06','07','08','09'] and not code.startswith('0'):
            return f"0{code}"
        return code

    v_read_Excel_df['CODE_LVL'] = def_level(v_read_Excel_df['CODE'])
    v_read_Excel_df['CODE'] = v_read_Excel_df.apply(lambda row: add_zero(row['CODE'], row['CODE_LVL']), axis=1)
    v_read_Excel_df['CODE']=v_read_Excel_df['CODE'].astype(str)
    #print(v_read_Excel_df)
    #with pd.ExcelWriter(path, engine='openpyxl', mode='a') as writer:
        #v_read_Excel_df.to_excel(writer, sheet_name='NewSheetName11', index=False)
    Merged_DF = pd.merge(COICOP_DF['CODE'],v_read_Excel_df, on='CODE', how='left')
    return Merged_DF[['CODE']+cpi_column_name]


#FUJ
try:
    if not os.path.exists(FUJ_path):
        raise FileNotFoundError(f"The file '{FUJ_path}' does not exist.")
    else:
        fuj_d_type = {key: str for key in FUJ_code_col_to_read}
        #print(fuj_d_type)
        FUJ_Merged=Read_Excel(FUJ_path,FUJ_code_col_to_read,FUJ_cpi_col_to_read,FUJ_start_row,FUJ_end_row,FUJ_cpi_column_name,fuj_d_type)
        FUJ_Merged_rows = FUJ_Merged.shape[0]
        Log_DF.loc[len(Log_DF)] = ['FUJ File:'+str(FUJ_Merged_rows)+' Rows Readed at', current_date]
        fuj_rd_flg=1
except FileNotFoundError as e:
    FUJ_Merged=pd.DataFrame()
    Log_DF.loc[len(Log_DF)] = [e, current_date]
    fuj_rd_flg=0

#RAK
try:
    if not os.path.exists(RAK_path):
        raise FileNotFoundError(f"The file '{RAK_path}' does not exist.")
    else:
        rak_d_type = {key: str for key in RAK_code_col_to_read}
        #print(rak_d_type)
        RAK_Merged=Read_Excel(RAK_path,RAK_code_col_to_read,RAK_cpi_col_to_read,RAK_start_row,RAK_end_row,RAK_cpi_column_name,rak_d_type)
        RAK_Merged_rows = RAK_Merged.shape[0]
        Log_DF.loc[len(Log_DF)] = ['RAK File:'+str(RAK_Merged_rows)+' Rows Readed at', current_date]
        rak_rd_flg=1
except FileNotFoundError as e:
    RAK_Merged=pd.DataFrame()
    Log_DF.loc[len(Log_DF)] = [e, current_date]
    fuj_rd_flg=0

#ABD
try:
    if not os.path.exists(ABD_path):
        raise FileNotFoundError(f"The file '{ABD_path}' does not exist.")
    else:
        abd_d_type = {key: str for key in ABD_code_col_to_read}
        #print(abd_d_type)
        ABD_Merged=Read_Excel(ABD_path,ABD_code_col_to_read,ABD_cpi_col_to_read,ABD_start_row,ABD_end_row,ABD_cpi_column_name,abd_d_type)
        ABD_Merged_rows = ABD_Merged.shape[0]
        Log_DF.loc[len(Log_DF)] = ['ABD File:'+str(ABD_Merged_rows)+' Rows Readed at', current_date]
        abd_rd_flg=1
except FileNotFoundError as e:
    ABD_Merged=pd.DataFrame()
    Log_DF.loc[len(Log_DF)] = [e, current_date]
    abd_rd_flg=0

#DXB
try:
    if not os.path.exists(DXB_path):
        raise FileNotFoundError(f"The file '{DXB_path}' does not exist.")
    else:
        dxb_d_type = {key: str for key in DXB_code_col_to_read}
        #print(dxb_d_type)
        DXB_Merged=Read_Excel(DXB_path,DXB_code_col_to_read,DXB_cpi_col_to_read,DXB_start_row,DXB_end_row,DXB_cpi_column_name,dxb_d_type)
        DXB_Merged_rows = DXB_Merged.shape[0]
        Log_DF.loc[len(Log_DF)] = ['DXB File:'+str(DXB_Merged_rows)+' Rows Readed at', current_date]
        dub_rd_flg=1
        #print(DXB_Merged)
except FileNotFoundError as e:
    DXB_Merged=pd.DataFrame()
    Log_DF.loc[len(Log_DF)] = [e, current_date]
    dub_rd_flg


print(Log_DF)
#print(Log_DF.iloc[[-1]])
#print(FUJ_Merged)
#print(RAK_Merged)
#print(DXB_Merged)
#print(ABD_Merged)

#FUJ_Merged_DF = pd.merge(COICOP_DF['CODE'],FUJ_Merged, on='CODE', how='left')
#RAK_Merged_DF = pd.merge(FUJ_Merged_DF,RAK_Merged, on='CODE', how='left')
#DXB_Merged_DF = pd.merge(RAK_Merged_DF,DXB_Merged, on='CODE', how='left')
#ABD_Merged_DF = pd.merge(DXB_Merged_DF,ABD_Merged, on='CODE', how='left')

list_DF=[FUJ_Merged,RAK_Merged,DXB_Merged,ABD_Merged]
Consoldtd_DF=COICOP_DF
for df in list_DF:
    if not df.empty:
        try:
            Consoldtd_DF = Consoldtd_DF.merge(df, on='CODE', how='left')
        except Exception as e:
            Log_DF.loc[len(Log_DF)] = ['Error while concat DF readed files: '+str(e), current_date]
            print('Error while concat DF readed files: '+str(e))
#print(Consoldtd_DF)

#===============================================Estimation =============================================
#v_UQ_AJ_SH_CPI_VAL=pd.DataFrame()
def Estimation_Phase(EST_FLG):
    if EST_FLG==1:
        global estimation_flg
        print("Estimation Process started...")
        try:
            #======================================================= Flaten Dubai and AbuDhabi Data ============================================
            def convrt_str_date(date_str):
                try:
                    month = date_str[:3]
                    year = date_str[4:6]
                    month_mapping = {
                        'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04',
                        'MAY': '05', 'JUN': '06', 'JUL': '07', 'AUG': '08',
                        'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}
                    year = '20' + year
                    #formatted_date = f"01-{month_mapping.get(month, '01')}-{year}"
                    #formatted_date = f"{year}-{month_mapping.get(month, '01')}-01"
                    date_str = f"{year}-{month_mapping.get(month, '12')}-01"
                    formatted_date = datetime.strptime(date_str, "%Y-%m-%d")
                    formatted_date_str = formatted_date.strftime("%Y-%m-%d")
                    return formatted_date_str
                except Exception as e:
                    print('Error while converting date: '+str(e))
                    Log_DF.loc[len(Log_DF)] = ['Error while converting date: '+str(e), current_date]
            def flaten_DF(AD_DU_DF):
                try:
                    #print('YUI',AD_DU_DF)
                    #print(AD_DU_DF.columns[1])
                    filtered_df = AD_DU_DF[AD_DU_DF['CODE'].str.len() == 4]
                    transposed_df = filtered_df.set_index('CODE').transpose()
                    new_column = pd.DataFrame({'date': []})
                    transposed_df.insert(0, 'date', [convrt_str_date(AD_DU_DF.columns[1])])
                    return transposed_df
                except Exception as e:
                    print('Error while flatenning AD, DU df: '+str(e), current_date)
                    Log_DF.loc[len(Log_DF)] = ['Error while flatenning AD, DU df: '+str(e), current_date]

            Flat_DXB=flaten_DF(DXB_Merged)
            #print(Flat_DXB)
            Flat_DXB.columns = ['isic' + col if col.lower() != 'date' else col for col in Flat_DXB.columns]
            Flat_ABD=flaten_DF(ABD_Merged)
            Flat_ABD.columns = ['isic' + col if col.lower() != 'date' else col for col in Flat_ABD.columns]
            #print(Flat_DXB)
            #print(Flat_ABD)
            #exit()  # or sys.exit()
            #=======================================================================================

            #print(Log_DF.iloc[[-1]])


            def upd_dubai_data(Path):
                #print('Upd Dubai')
                try:
                    UQ_Check_Du =pd.read_excel(Path, sheet_name='DU', engine='openpyxl') #dtype=str
                    UQ_Check_Du['date'] = pd.to_datetime(UQ_Check_Du['date']).dt.strftime('%Y-%m-%d')
                    #v_Chk_max_date=datetime.strptime(str(UQ_Check_Du.iloc[:,0].max())[0:10], '%Y-%m-%d')
                    #UQ_Check_Du.iloc[:, 0] = pd.to_datetime(UQ_Check_Du.iloc[:, 0],format='%Y-%m-%d') # infer_datetime_format=True) #format='%d/%m/%Y %I:%M:%S %p')
                    v_Chk_max_date = UQ_Check_Du.iloc[:, 0].max()
                    #v_curr_date=datetime.strptime(str(Flat_DXB.iloc[:,0].max()), '%Y-%m-%d')
                    #Flat_DXB.iloc[:, 0] = pd.to_datetime(Flat_DXB.iloc[:, 0],format='%Y-%m-%d') # infer_datetime_format=True) #format='%d/%m/%Y %I:%M:%S %p')
                    v_curr_date = Flat_DXB.iloc[:, 0].max()
                    #print(v_Chk_max_date)
                    #print(v_curr_date)
                    if v_Chk_max_date<v_curr_date:
                        updated_data = pd.concat([UQ_Check_Du, Flat_DXB], ignore_index=True)
                        #book = load_workbook(Path)
                        #sheet_name = 'DU'
                        #sheet = book[sheet_name]
                        #last_row = sheet.max_row
                        #for col_idx, value in enumerate(Flat_DXB.iloc[0], start=1):
                            #sheet.cell(row=last_row + 1, column=col_idx, value=value)
                        #book.save(Path)
                        #book.close()
                        with pd.ExcelWriter(Path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                updated_data.to_excel(writer, index=False, sheet_name='DU')
                        print("\n Dubai data updated to sheet DU in the Excel file.\n")
                        Log_DF.loc[len(Log_DF)] = ['Dubai data updated to sheet DU in the Excel file.', current_date]
                        return 1
                    else:
                        if v_Chk_max_date==v_curr_date:
                            print("\n Dubai data of date: "+str(Flat_DXB.iloc[:,0].max())+" Already  in sheet DU in the Excel file.\n")
                            Log_DF.loc[len(Log_DF)] = ['Dubai data of date: '+str(Flat_DXB.iloc[:,0].max())+' Already  in sheet DU in the Excel file.', current_date]
                            return 1
                        else:
                            return 0
                except Exception as e:
                    print(str(e))

            def upd_Abu_data(Path):
                #print('i')
                try:
                    UQ_Check_AD =pd.read_excel(Path, sheet_name='AD', engine='openpyxl') #dtype=str
                    UQ_Check_AD['date'] = pd.to_datetime(UQ_Check_AD['date']).dt.strftime('%Y-%m-%d')
                    #print('CHAK_DATE',UQ_Check_AD)
                    #v_Chk_max_date=datetime.strptime(str(UQ_Check_AD.iloc[:,0].max())[0:10], '%Y-%m-%d')
                    #UQ_Check_AD.iloc[:, 0] = pd.to_datetime(UQ_Check_AD.iloc[:, 0],format='%Y-%m-%d') # infer_datetime_format=True) #format='%d/%m/%Y %I:%M:%S %p')
                    v_Chk_max_date = UQ_Check_AD.iloc[:, 0].max()
                   # print(v_Chk_max_date)
                    #v_curr_date=datetime.strptime(str(Flat_ABD.iloc[:,0].max()), '%Y-%m-%d')
                    #Flat_ABD.iloc[:, 0] = pd.to_datetime(Flat_ABD.iloc[:, 0],format='%Y-%m-%d') # infer_datetime_format=True) #format='%d/%m/%Y %I:%M:%S %p')
                    v_curr_date = Flat_ABD.iloc[:, 0].max()
                    #print(v_Chk_max_date)
                    #print(v_curr_date)
                    if v_Chk_max_date<v_curr_date:
                        updated_data = pd.concat([UQ_Check_AD, Flat_ABD], ignore_index=True)
                        #print('AD',Flat_ABD)
                        #print(Flat_ABD, pd.DataFrame)
                        #book = load_workbook(Path)
                        #sheet_name = 'AD'  # Change this to your actual sheet name
                        #sheet = book[sheet_name]
                        #last_row = sheet.max_row
                        #for col_idx, value in enumerate(Flat_ABD.iloc[0], start=1):  # Use iloc[0] to get the first (and only) row
                            #sheet.cell(row=last_row + 1, column=col_idx, value=value)
                        #book.save(Path)
                        #book.close()
                        with pd.ExcelWriter(Path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                                updated_data.to_excel(writer, index=False, sheet_name='AD')
                        print("\n Abudhabi data updated to sheet AD in the Excel file.\n")
                        Log_DF.loc[len(Log_DF)] = ['Abudhabi data updated to sheet AD in the Excel file.', current_date]
                        return 1
                    else:
                        if v_Chk_max_date==v_curr_date:
                            print("\n Abudhabi data of date: "+str(Flat_ABD.iloc[:,0].max())+" Already  in sheet AD in the Excel file.\n")
                            Log_DF.loc[len(Log_DF)] = ['Abudhabi data of date: '+str(Flat_ABD.iloc[:,0].max())+' Already  in sheet AD in the Excel file.', current_date]
                            return 1
                        else:
                            return 0
                except Exception as e:
                    print(str(e))


            # ==================================== Start Estimations ============================================================


            # Define the file path
            #file_path = "C:/Users/SSMEPZ/3D Objects/CP/CPI/MissingEmirate_Calc/TEST/bdd_UQ.xlsx"
            #Sheet_Name="UQ"
            def ESTIMATIONS(file_path,Sheet_Name,Temp_pred):
                # Load the data
                try:
                    pandas2ri.deactivate()
                    pandas2ri.activate()
                    in_file_path=file_path.replace("\\", "/")
                    temp_path=Temp_pred
                    Emirate = pd.read_excel(file_path, sheet_name=Sheet_Name)
                    abd = pd.read_excel(file_path, sheet_name="AD")
                    ad= abd[:-1]
                    #ad= abd
                    dub = pd.read_excel(file_path, sheet_name="DU")
                    du= dub[:-1]
                    #du= dub
                    #ad_test = pd.read_excel(file_path, sheet_name="AD_test")
                    ad_test=abd.iloc[-2:]
                    #du_test = pd.read_excel(file_path, sheet_name="DU_test")
                    du_test=dub.iloc[-2:]
                    #print(Emirate)
                    #print(ad)
                    #print(du)
                    #print(ad_test)
                    #print(du_test)

                    Emirate_r = pandas2ri.py2rpy(Emirate)
                    ad_r = pandas2ri.py2rpy(ad)
                    du_r = pandas2ri.py2rpy(du)
                    ad_test_r = pandas2ri.py2rpy(ad_test)
                    du_test_r = pandas2ri.py2rpy(du_test)
                    #path_r = pandas2ri.py2rpy(in_file_path)
                    robjects.globalenv['aj'] = Emirate_r
                    robjects.globalenv['ad'] = ad_r
                    robjects.globalenv['du'] = du_r
                    robjects.globalenv['ad_test'] = ad_test_r
                    robjects.globalenv['du_test'] = du_test_r

                    #r.assign('aj', Emirate_r)
                    #r.assign('ad', ad_r)
                    #r.assign('du', du_r)
                    #r.assign('ad_test', ad_test_r)
                    #r.assign('du_test', du_test_r)
                    robjects.globalenv['file_path'] = robjects.StrVector([in_file_path])
                    robjects.globalenv['temp_pred_path'] = robjects.StrVector([temp_path])

                    # To remove Predictions sheet


                    r_code = """
                    # Load necessary libraries
                    #install.packages("openxlsx")
                    #install.packages("dplyr")
                    if (!requireNamespace("dplyr", quietly = TRUE)) install.packages("dplyr")
                    if (!requireNamespace("openxlsx", quietly = TRUE)) install.packages("openxlsx")
                    library(dplyr)
                    library(openxlsx)
                    #print(aj)
                    #print(ad)
                    #print(du)
                    #print(ad_test)
                    #print(du_test)
                    # Function to calculate log differences row-wise for each ISIC variable
                    log_diff <- function(df) {
                      df_diff <- df
                      for (i in 2:nrow(df)) {
                        for (j in 2:ncol(df)) {
                          if (!is.na(df[i, j]) && !is.na(df[i-1, j]) && df[i-1, j] != 0) {
                            df_diff[i, j] <- log(df[i, j] / df[i-1, j])
                          } else {
                            df_diff[i, j] <- NA
                          }
                        }
                      }
                      return(df_diff[-1, ])  # Remove the first row as it will have NAs
                    }

                    # Calculate log differences for AJ, DU, AD, AD_test, and DU_test
                    aj_log_diff <- log_diff(aj)
                    du_log_diff <- log_diff(du)
                    ad_log_diff <- log_diff(ad)
                    du_test_log_diff <- log_diff(du_test)
                    ad_test_log_diff <- log_diff(ad_test)

                    # Perform regression analysis for each ISIC code and make predictions
                    isic_codes <- colnames(aj)[-1]  # Exclude the 'date' column

                    predictions <- list()

                    for (code in isic_codes) {
                      # Prepare the data for log_diff_AJ ~ log_diff_DU + log_diff_AD
                      data <- data.frame(
                        AJ = aj_log_diff[[code]],
                        DU = du_log_diff[[code]],
                        AD = ad_log_diff[[code]]
                      )

                      # Remove rows with NA values
                      data <- na.omit(data)

                      # Check if data is empty after removing NAs
                      if (nrow(data) > 0) {
                        # Perform the regression
                        model <- lm(AJ ~ DU + AD, data = data)
                        model_D <- lm(AJ ~ DU, data = data)
                        model_AD <- lm(AJ ~ AD, data = data)

                        # Check significance of all coefficients in the full model
                        summary_model <- summary(model)
                        p_values <- summary_model$coefficients[, 4]

                        #############################################################
                        # Prepare the test data
                        test_data <- data.frame(
                          DU = du_test_log_diff[[code]],
                          AD = ad_test_log_diff[[code]]
                        )

                        # Remove rows with NA values
                        test_data <- na.omit(test_data)
                        test_data_du <- test_data[ , "DU", drop = FALSE]
                        test_data_ad <- test_data[ , "AD", drop = FALSE]

                        # Make predictions based on model selection
                        if (nrow(test_data) > 0) {
                          if (!is.na(p_values[2] <= 0.05) && !is.na(p_values[3] <= 0.05)) {
                            # Use the full model if any coefficient is significant
                            pred <- predict(model, newdata = test_data)
                          } else {
                            # Compare model_D and model_AD based on AIC (or another metric)
                            aic_D <- AIC(model_D)
                            aic_AD <- AIC(model_AD)

                            if (aic_D < aic_AD) {
                              pred <- predict(model_D, newdata = test_data_du)
                            } else {
                              pred <- predict(model_AD, newdata = test_data_ad)
                            }
                          }
                          predictions[[code]] <- pred
                        } else {
                          predictions[[code]] <- NA
                          cat("\nNot enough test data for ISIC code:", code, "after removing NAs\n")
                        }

                      }

                      # Check for unchanged CPI row-wise
                      if (abs(du_test[2, code] - du_test[1, code]) < 0.04) {
                        predictions[[code]] <- 0
                      }
                    }

                    # Ensure all ISIC codes are included in the prediction sheet
                    all_codes <- colnames(du)[-1]
                    predictions_df <- as.data.frame(matrix(NA, nrow = nrow(du_test_log_diff), ncol = length(all_codes)))
                    colnames(predictions_df) <- all_codes
                    for (code in all_codes) {
                      if (code %in% names(predictions)) {
                        predictions_df[[code]] <- predictions[[code]]
                      }
                    }
                    predictions_df <- cbind(date = du_test_log_diff[, 1], predictions_df)
                    #file_path="C:/Users/SSMEPZ/3D Objects/CP/CPI/CPI Automation/CPI_Input_Files/Estimation_Input_Files/bdd_AJ_.xlsx"
                    # Load the original workbook
                    #print(predictions_df)
                    wb <- loadWorkbook(temp_pred_path)
                    removeWorksheet(wb, "Predictions")
                    # Write predictions to a new sheet
                    addWorksheet(wb, "Predictions")
                    writeData(wb, "Predictions", predictions_df)

                    # Save the workbook
                    saveWorkbook(wb, temp_pred_path, overwrite = TRUE)

                    cat("\nPredictions saved to 'Predictions' sheet in the Excel file.\n")
                    """
                    robjects.r(r_code)
                    predictions_df= pd.read_excel(Temp_pred_path, sheet_name='Predictions')
                    #print('Readed Pred',predictions_df)
                    #print("\nPredictions saved to 'Predictions' sheet in the Excel file.\n")
                    last_row = Emirate.iloc[-1:]
                    #print(last_row)
                    combined_df = pd.concat([last_row, predictions_df], axis=0, ignore_index=True)
                    combined_df = combined_df.fillna(0)
                    #print(combined_df)
                    isic_Combined = combined_df.columns[1:]
                    combined_row = {'date': combined_df['date'].iloc[1]}
                    for code in isic_Combined:
                        code1='isic0623'
                        B1 = combined_df[code].iloc[0]
                        B2 = combined_df[code].iloc[1]
                        result_value = math.exp(B2) * B1
                        combined_row[code] = result_value
                    out_df = pd.DataFrame([combined_row])
                    #print('Emirates',Emirate)
                    #print('OutDF',out_df)
                    out_df['date'] = pd.to_datetime(out_df['date']).dt.strftime('%Y-%m-%d')
                    Emirate['date'] = pd.to_datetime(Emirate['date']).dt.strftime('%Y-%m-%d')
                    Max_date=str(Emirate.iloc[:, 0].max())
                    Curr_Date=str(out_df.iloc[:, 0].max())
                    #print('Est',Max_date)
                    #print('Est',Curr_Date)
                    #test = pd.concat([combined_df, out_df], axis=0, ignore_index=True)
                    if Max_date!=Curr_Date:
                        #with pd.ExcelWriter(file_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
                            #out_df.to_excel(writer, sheet_name='Predictions', index=False)
                        #print("Predictions saved to 'Predictions' sheet in the Excel file")
                        #Log_DF.loc[len(Log_DF)] = ['Predictions saved to Predictions sheet in the Excel file.', current_date]
                        updated_data = pd.concat([Emirate, out_df], ignore_index=True)
                        #book = load_workbook(file_path)
                        #sheet_name = Sheet_Name
                        #sheet = book[sheet_name]
                        #last_row = sheet.max_row
                        #for col_idx, value in enumerate(out_df.iloc[0], start=1):
                            #sheet.cell(row=last_row + 1, column=col_idx, value=value)
                        #book.save(file_path)
                        #book.close()

                        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                            updated_data.to_excel(writer, index=False, sheet_name=Sheet_Name)
                            #print("Predictions Updated to"+Sheet_Name+"sheet in the Excel file")
                        Log_DF.loc[len(Log_DF)] = ['Predictions Updated to '+Sheet_Name+' sheet in the Excel file.', current_date]
                        print('Predictions Updated to '+Sheet_Name+' sheet in the Excel file.')
                            #print(updated_data)
                        return 1
                    else:
                        print('Predictions for month '+Curr_Date+' already exist')
                        Log_DF.loc[len(Log_DF)] = ['Predictions for month '+Curr_Date+' already exist', current_date]
                        print('Predictions for month '+Curr_Date+' already exist')
                        return 0
                except Exception as e:
                    print(str(e))
            #==================================Estimations Start ============================================================
            Estimation_start_flg=0
            Est_UQ_Flg=0
            Est_SH_Flg=0
            Est_AJ_Flg=0

            path=[UMQ_path,AJ_path,SH_path]
            sheet=[UMQ_Sheet_Name,AJ_Sheet_Name,SH_Sheet_Name]
            #temp=[Temp_pred_path,Temp_pred_path,Temp_pred_path]
            #print('Est')
            if upd_Abu_data(AJ_path)==1 and upd_dubai_data(AJ_path)==1 and upd_Abu_data(UMQ_path)==1 and upd_dubai_data(UMQ_path)==1 and upd_Abu_data(SH_path)==1 and upd_dubai_data(SH_path)==1:
                Estimation_start_flg=1
                Log_DF.loc[len(Log_DF)] = ['Abudhabi, Dubai data loaded to Estimation input files', current_date]
                print('Abudhabi, Dubai data loaded to Estimation input files')
            else:
                Estimation_start_flg=0
                Log_DF.loc[len(Log_DF)] = ['Abudhabi, Dubai not loaded to Estimations input files', current_date]
                print('Abudhabi, Dubai not loaded to Estimations input files')
            if Estimation_start_flg==1:
                for i,j in zip(path,sheet):
                    if j=='UQ':
                        if ESTIMATIONS(i,j,Temp_pred_path)==1:
                            Est_UQ_Flg=1
                            Log_DF.loc[len(Log_DF)] = ['UQ Estimation Completed', current_date]
                            print('UQ Estimation Completed')
                        else:
                            Log_DF.loc[len(Log_DF)] = ['UQ Estimation Not Completed', current_date]
                            print('UQ Estimation Not Completed')
                    elif j=='AJ':
                        if ESTIMATIONS(i,j,Temp_pred_path)==1:
                            Est_AJ_Flg=1
                            Log_DF.loc[len(Log_DF)] = ['AJ Estimation Completed', current_date]
                            print('AJ Estimation Completed')
                        else:
                            Log_DF.loc[len(Log_DF)] = ['AJ Estimation Not Completed', current_date]
                            print('AJ Estimation Not Completed')
                    elif j=='SH':
                        if ESTIMATIONS(i,j,Temp_pred_path)==1:
                            Est_SH_Flg=1
                            Log_DF.loc[len(Log_DF)] = ['SH Estimation Completed', current_date]
                            print('SH Estimation Completed')
                        else:
                            Log_DF.loc[len(Log_DF)] = ['SH Estimation Not Completed', current_date]
                            print('SH Estimation Not Completed')

            else:
                print('End the excecution because of AD, DU data not updated to Estimations input files')
                Log_DF.loc[len(Log_DF)] = ['End the excecution because of AD, DU data not updated to Estimations input files', current_date]

            if Est_UQ_Flg==1 and Est_AJ_Flg==1 and Est_SH_Flg==1:
                UQ = pd.DataFrame()
                AJ = pd.DataFrame()
                SH = pd.DataFrame()

                select_col=['Code']

                def Cal_Estimations(path,sheet_name):
                    Emi=['SH','AJ','UQ']
                    if j in Emi:
                        read= pd.read_excel(path, dtype=str, engine='openpyxl')
                        last_row=read.iloc[-1:]
                        Normalized_DF= pd.melt(last_row, id_vars=['date'], var_name='Code', value_name=j+'_value')
                    else:
                        print("Sheet Name Should be emiarte code -  AJ , SH, UQ")
                        Log_DF.loc[len(Log_DF)] = ['Sheet Name Should be emiarte code -  AJ , SH, UQ', current_date]
                        #print('Sheet Name Should be emiarte code -  AJ , SH, UQ')
                    return Normalized_DF

                for i,j in zip(path,sheet):
                    if j=='UQ':
                        UQ=Cal_Estimations(i,j)
                        select_col.append(j+'_value')
                    elif j=='AJ':
                        AJ=Cal_Estimations(i,j)
                        select_col.append(j+'_value')
                    elif j=='SH':
                        SH=Cal_Estimations(i,j)
                        select_col.append(j+'_value')

                v_DF1 = pd.merge(UQ,AJ, on='Code', how='inner')
                v_DF2 = pd.merge(v_DF1,SH, on='Code', how='inner')
                #print(select_col)
                Est_Emirates=v_DF2[[col for col in select_col]]
                Est_Emirates.loc[:,'Code'] = Est_Emirates['Code'].str.replace('isic', '', regex=False)
                #print(Est_Emirates)

                #===============RAED Est. Emirates Weightage================================
                Est_Emirate_Weight = pd.read_excel(Weight_path, dtype={'Code': str, 'Three_Code': str, 'Two_Code': str}, engine='openpyxl' )
                #Est_weight_col=['UQ_LOW','AJ_LOW','SH_LOW','UQ_HIGH','AJ_HIGH','SH_HIGH']
                #for i in Est_weight_col:
                    #Est_Emirate_Weight[i] = pd.to_numeric(Est_Emirate_Weight[i], errors='coerce')
                #print(Est_Emirate_Weight)

                #==================================================
                Four_Digt_Val= pd.merge(Est_Emirates,Est_Emirate_Weight, on='Code', how='inner')

                four_df_col=['UQ_value','AJ_value','SH_value','UQ_LOW','AJ_LOW','SH_LOW','SH_THRE_W','UQ_THRE_W','AJ_THRE_W']

                for i in four_df_col:
                    Four_Digt_Val[i] = pd.to_numeric(Four_Digt_Val[i], errors='coerce')


                #============Alcoholic_Beverages_And_Tobacco_three_codes
                NA_Four_digit_codes=['0211']
                Four_Digt_Val.loc[Four_Digt_Val['Code'].isin(NA_Four_digit_codes), 'SH_value'] = np.nan

                # =====================3 Digit CPI Calculation ===============================================================
                #Codes that has single 4 Level
                try:
                    mask_req_thr_codes = Four_Digt_Val['Three_Code'].isin(['021','023','032','041','042','052','054','063','064','082','091','094','098','101','102','104','105','112','122','133','139'])

                    Four_Digt_Val.loc[mask_req_thr_codes,'WEIGHT_MUL_UQ'] = (Four_Digt_Val['UQ_value'] * Four_Digt_Val['UQ_THRE_W'])/100
                    Four_Digt_Val.loc[mask_req_thr_codes,'WEIGHT_MUL_AJ'] = (Four_Digt_Val['AJ_value'] * Four_Digt_Val['AJ_THRE_W'])/100
                    Four_Digt_Val.loc[mask_req_thr_codes,'WEIGHT_MUL_SH'] = (Four_Digt_Val['SH_value'] * Four_Digt_Val['SH_THRE_W'])/100

                    Four_Digt_Val.loc[~mask_req_thr_codes,'WEIGHT_MUL_UQ'] = (Four_Digt_Val['UQ_value'] * Four_Digt_Val['UQ_LOW'])/100
                    Four_Digt_Val.loc[~mask_req_thr_codes,'WEIGHT_MUL_AJ'] = (Four_Digt_Val['AJ_value'] * Four_Digt_Val['AJ_LOW'])/100
                    Four_Digt_Val.loc[~mask_req_thr_codes,'WEIGHT_MUL_SH'] = (Four_Digt_Val['SH_value'] * Four_Digt_Val['SH_LOW'])/100
                except Exception as e:
                    print(str(e))

                v_three_UQ = Four_Digt_Val.groupby('Three_Code', as_index=False)['WEIGHT_MUL_UQ'].sum()
                v_three_UQ.columns=['Code','WEIGHT_MUL_UQ']

                v_three_AJ = Four_Digt_Val.groupby('Three_Code', as_index=False)['WEIGHT_MUL_AJ'].sum()
                v_three_AJ.columns=['Code','WEIGHT_MUL_AJ']

                v_three_SH = Four_Digt_Val.groupby('Three_Code', as_index=False)['WEIGHT_MUL_SH'].sum()
                v_three_SH.columns=['Code','WEIGHT_MUL_SH']

                v_three_df= pd.merge(v_three_UQ,v_three_AJ, on='Code', how='inner')
                v1_three_df= pd.merge(v_three_df,v_three_SH, on='Code', how='inner')



                v2_three_df= pd.merge(v1_three_df,Est_Emirate_Weight, on='Code', how='inner')
                #print(v2_three_df)
                #print('v2_three_df',v2_three_df)

                v2_three_df['CPI_UQ']=(v2_three_df['WEIGHT_MUL_UQ'] / v2_three_df['UQ_LOW'])*100
                v2_three_df['CPI_AJ']=(v2_three_df['WEIGHT_MUL_AJ'] / v2_three_df['AJ_LOW'])*100
                v2_three_df['CPI_SH']=(v2_three_df['WEIGHT_MUL_SH'] / v2_three_df['SH_LOW'])*100

                Three_code_col=['Code','Two_Code','CPI_UQ','CPI_AJ','CPI_SH']
                three_dig_CPI=v2_three_df[Three_code_col]

                #print(three_dig_CPI)

                #============Alcoholic_Beverages_And_Tobacco_three_codes
                #NA_Three_digit_codes=['021','023']
                #three_dig_CPI.loc[three_dig_CPI['Code'].isin(NA_Three_digit_codes), 'CPI_SH'] = np.nan

                #print(three_dig_CPI)

                #=================================3 Digit CPI calc END ====================================================

                #==================================2 digit Calc Start =========================================

                v_two_Digt_Val= pd.merge(three_dig_CPI[['Code','CPI_UQ','CPI_AJ','CPI_SH']],Est_Emirate_Weight, on='Code', how='inner')
                #print(v_two_Digt_Val)

                v_two_Digt_Val['WEIGHT_MUL_UQ'] = (v_two_Digt_Val['CPI_UQ'] * v_two_Digt_Val['UQ_LOW'])/100
                v_two_Digt_Val['WEIGHT_MUL_AJ'] = (v_two_Digt_Val['CPI_AJ'] * v_two_Digt_Val['AJ_LOW'])/100
                v_two_Digt_Val['WEIGHT_MUL_SH'] = (v_two_Digt_Val['CPI_SH'] * v_two_Digt_Val['SH_LOW'])/100

                #print(v_two_Digt_Val)
                v_two_UQ = v_two_Digt_Val.groupby('Two_Code', as_index=False)['WEIGHT_MUL_UQ'].sum()
                v_two_UQ.columns=['Code','WEIGHT_MUL_UQ']
                #print(v_two_UQ)
                v_two_AJ = v_two_Digt_Val.groupby('Two_Code', as_index=False)['WEIGHT_MUL_AJ'].sum()
                v_two_AJ.columns=['Code','WEIGHT_MUL_AJ']

                v_two_SH = v_two_Digt_Val.groupby('Two_Code', as_index=False)['WEIGHT_MUL_SH'].sum()
                v_two_SH.columns=['Code','WEIGHT_MUL_SH']

                v_two_df= pd.merge(v_two_UQ,v_two_AJ, on='Code', how='inner')
                v1_two_df= pd.merge(v_two_df,v_two_SH, on='Code', how='inner')

                #print(v1_two_df)
                v2_two_df= pd.merge(v1_two_df,Est_Emirate_Weight, on='Code', how='inner')

                #print(v2_two_df)
                v2_two_df[prc_month+'_'+prc_year+'_UQ']=(v2_two_df['WEIGHT_MUL_UQ'] / v2_two_df['UQ_LOW'])*100
                v2_two_df[prc_month+'_'+prc_year+'_AJ']=(v2_two_df['WEIGHT_MUL_AJ'] / v2_two_df['AJ_LOW'])*100
                v2_two_df[prc_month+'_'+prc_year+'_SH']=(v2_two_df['WEIGHT_MUL_SH'] / v2_two_df['SH_LOW'])*100

                #print(v2_two_df)

                two_code_col=['Code',prc_month+'_'+prc_year+'_UQ',prc_month+'_'+prc_year+'_AJ',prc_month+'_'+prc_year+'_SH']
                two_dig_CPI=v2_two_df[two_code_col]

                #print(two_dig_CPI)

                #===================Two Digit CPI Calc END =================================

                #===================== '00' Level CPI calc Start ===========================

                #print(two_dig_CPI)
                v_zero_Digt_Val= pd.merge(two_dig_CPI[['Code',prc_month+'_'+prc_year+'_UQ',prc_month+'_'+prc_year+'_AJ',prc_month+'_'+prc_year+'_SH']],Est_Emirate_Weight, on='Code', how='inner')


                v_zero_Digt_Val['WEIGHT_MUL_UQ'] = (v_zero_Digt_Val[prc_month+'_'+prc_year+'_UQ'] * v_zero_Digt_Val['UQ_LOW'])/100
                v_zero_Digt_Val['WEIGHT_MUL_AJ'] = (v_zero_Digt_Val[prc_month+'_'+prc_year+'_AJ'] * v_zero_Digt_Val['AJ_LOW'])/100
                v_zero_Digt_Val['WEIGHT_MUL_SH'] = (v_zero_Digt_Val[prc_month+'_'+prc_year+'_SH'] * v_zero_Digt_Val['SH_LOW'])/100

                #print(v_zero_Digt_Val)

                v_total_UQ = v_zero_Digt_Val['WEIGHT_MUL_UQ'].sum()
                #total_UQ=(v_total_UQ/0.009065)*100
                total_UQ=(v_total_UQ/1.0)*100
                v_zero_UQ = pd.DataFrame({'Code': ['00'], prc_month+'_'+prc_year+'_UQ': [total_UQ]})

                v_total_AJ = v_zero_Digt_Val['WEIGHT_MUL_AJ'].sum()
                #total_AJ=(v_total_AJ/0.055441)*100
                total_AJ=(v_total_AJ/1.0)*100
                v_zero_AJ = pd.DataFrame({'Code': ['00'], prc_month+'_'+prc_year+'_AJ': [total_AJ]})

                v_total_SH = v_zero_Digt_Val['WEIGHT_MUL_SH'].sum()
                #total_SH=(v_total_SH/0.180239)*100
                total_SH=(v_total_SH/1.0)*100

                v_zero_SH = pd.DataFrame({'Code': ['00'], prc_month+'_'+prc_year+'_SH': [total_SH]})

                v_zero_df= pd.merge(v_zero_UQ,v_zero_AJ, on='Code', how='inner')
                v1_zero_df= pd.merge(v_zero_df,v_zero_SH, on='Code', how='inner')

                #print(v1_zero_df)

                #=================================Zero Level CPI Calc END =============================

                #======================= PRINT ALL LEVELS CPI =========================================

                out_four_code_cpi=Four_Digt_Val[['Code','UQ_value','AJ_value','SH_value']]
                out_four_code_cpi.columns=['Code',prc_month+'_'+prc_year+'_UQ',prc_month+'_'+prc_year+'_AJ',prc_month+'_'+prc_year+'_SH']
                #print(out_four_code_cpi)
                out_three_code_cpi=three_dig_CPI[['Code','CPI_UQ','CPI_AJ','CPI_SH']]
                out_three_code_cpi.columns=['Code',prc_month+'_'+prc_year+'_UQ',prc_month+'_'+prc_year+'_AJ',prc_month+'_'+prc_year+'_SH']
                v_UQ_AJ_SH_CPI_VAL= pd.concat([v1_zero_df, two_dig_CPI, out_three_code_cpi,out_four_code_cpi], ignore_index=True)
                estimation_flg=1
                #print(estimation_flg,'EST')
                return v_UQ_AJ_SH_CPI_VAL
            else:
                print('Error while AJ, SH, UQ predictions estimation')
                Log_DF.loc[len(Log_DF)] = ['Error while AJ, SH, UQ predictions estimation', current_date]
                #sys.exist()
        except Exception as e:
            Log_DF.loc[len(Log_DF)] = ['Estimation Phase '+str(e), current_date]
    else:
        print("Not Enough Emirates readed for CPI estimation")
        Log_DF.loc[len(Log_DF)] = ['Not Enough Emirates readed for CPI estimation', current_date]
#=======================================================================================

def Calc_CPI(DF,E_Weightage):
    print('CPI Calculation Started..')
    try:
        Readed_col=[prc_month+'_'+prc_year+'_FUJ'
        ,prc_month+'_'+prc_year+'_AJ'
        ,prc_month+'_'+prc_year+'_RAK'
        ,prc_month+'_'+prc_year+'_UQ'
        ,prc_month+'_'+prc_year+'_ABU'
        ,prc_month+'_'+prc_year+'_DUB'
        ,prc_month+'_'+prc_year+'_SH']
        for column in Readed_col:
            DF[column] = pd.to_numeric(DF[column], errors='coerce')

        #print(DF)

        v_Sum_of_CPI=pd.merge(DF,E_Weightage, on='CODE', how='inner')
        #print(v_Sum_of_CPI)
        v_Sum_of_CPI['WEIGHTED_SUM'] = (
            (v_Sum_of_CPI[prc_month+'_'+prc_year+'_FUJ'].fillna(0) * v_Sum_of_CPI['FUJ_WEIGHT'])/100 +
            (v_Sum_of_CPI[prc_month+'_'+prc_year+'_AJ'].fillna(0) * v_Sum_of_CPI['AJM_WEIGHT'])/100 +
            (v_Sum_of_CPI[prc_month+'_'+prc_year+'_RAK'].fillna(0) * v_Sum_of_CPI['RAK_WEIGHT'])/100 +
            (v_Sum_of_CPI[prc_month+'_'+prc_year+'_UQ'].fillna(0) * v_Sum_of_CPI['UQM_WEIGHT'])/100 +
        	(v_Sum_of_CPI[prc_month+'_'+prc_year+'_ABU'].fillna(0) * v_Sum_of_CPI['ABU_WEIGHT'])/100 +
        	(v_Sum_of_CPI[prc_month+'_'+prc_year+'_DUB'].fillna(0) * v_Sum_of_CPI['DUB_WEIGHT'])/100 +
        	(v_Sum_of_CPI[prc_month+'_'+prc_year+'_SH'].fillna(0) * v_Sum_of_CPI['SHJ_WEIGHT'])/100
        )

        #print(v_Sum_of_CPI)

        #v_Sum_of_CPI[prc_month+'_'+prc_year+'_UAE']=(v_Sum_of_CPI['WEIGHTED_SUM']/(v_Sum_of_CPI['STATE_WEIGHT_BY_STATE_TO_HOUSEHOLDS'])*100)
        #MUL_EMI_WEIGHTS
        v_Sum_of_CPI[prc_month+'_'+prc_year+'_UAE']=(v_Sum_of_CPI['WEIGHTED_SUM']/(v_Sum_of_CPI['ALL_EMI_MUL_WEIGHT'])*100)
        #print(v_Sum_of_CPI)
        #Out_CPI=Sum_of_CPI[['CODE','UAE']]
        Out_DF=v_Sum_of_CPI[['CODE',prc_month+'_'+prc_year+'_FUJ',prc_month+'_'+prc_year+'_AJ',prc_month+'_'+prc_year+'_RAK',prc_month+'_'+prc_year+'_UQ',prc_month+'_'+prc_year+'_ABU',prc_month+'_'+prc_year+'_DUB',prc_month+'_'+prc_year+'_SH',prc_month+'_'+prc_year+'_UAE']]
        columns_to_check = [col for col in Out_DF.columns if col != prc_month+'_'+prc_year+'_UAE' and 'CODE']
        def update_total(row):
            if row[columns_to_check].isna().all():
                return np.nan
            return row[prc_month+'_'+prc_year+'_UAE']
        Out_DF.loc[:,prc_month+'_'+prc_year+'_UAE'] = Out_DF.apply(update_total, axis=1)
        #print(Out_DF)
        zero_lvl_weight=pd.merge(Out_DF,E_Weightage, on='CODE', how='inner')
        v2_zero_Lvl_Cal=zero_lvl_weight[zero_lvl_weight['CODE'].isin(['01','02','03','04','05','06','07','08','09','10','11','12','13'])][['CODE',prc_month+'_'+prc_year+'_UAE','GLOBALE']]
        v2_zero_Lvl_Cal['MUL_EMI_VALUE'] = ((v2_zero_Lvl_Cal[prc_month+'_'+prc_year+'_UAE'].fillna(0) * v2_zero_Lvl_Cal['GLOBALE']))
        sum_of_emi_val=v2_zero_Lvl_Cal['MUL_EMI_VALUE'].sum()
        gloabl_weight_zero_lvl=v_Sum_of_CPI.loc[v_Sum_of_CPI['CODE'] =='00', 'GLOBALE'].loc[0]
        zero_CODE_val=sum_of_emi_val/gloabl_weight_zero_lvl
        Out_DF.loc[Out_DF['CODE'] == '00', prc_month+'_'+prc_year+'_UAE'] = zero_CODE_val
        print(Out_DF)
        print('CPI Calculations Completed................')
        Log_DF.loc[len(Log_DF)] = ['CPI Calculations Completed', current_date]
        return Out_DF
    except Exception as e:
        print('CPI calculation Failed : '+str(e))
        Log_DF.loc[len(Log_DF)] = ['CPI calculation Failed '+str(e), current_date]


Out_EMIRATE_CPI=pd.DataFrame()
UQ_AJ_SH_CPI_VAL=pd.DataFrame()
if coicop_flg==1 and emi_weight_flg==1 and fuj_rd_flg==1 and rak_rd_flg==1 and abd_rd_flg==1 and dub_rd_flg==1:
    UQ_AJ_SH_CPI_VAL=Estimation_Phase(1)
    #print(estimation_flg,'EST')
    #print(UQ_AJ_SH_CPI_VAL)
    if estimation_flg==1:
        UQ_AJ_SH_CPI_VAL.rename(columns={'Code': 'CODE'}, inplace=True)
        Con_All_Emi_CPI = pd.merge(Consoldtd_DF, UQ_AJ_SH_CPI_VAL, on='CODE', how='outer')
        print(Con_All_Emi_CPI)
        Out_EMIRATE_CPI=Calc_CPI(Con_All_Emi_CPI,Emirate_Weightage)
        arch_flg=1
else:
    print('Error While reading input files')
    Log_DF.loc[len(Log_DF)] = ['Error While reading input files', current_date]

#print(Out_EMIRATE_CPI)

def ARCH_PRC(arch_flag):
    if arch_flag==1:
        print('Started wrting to ARCH files')
        try:
            ARCH_path=Output_Path
            start_row=1
            end_row=170
            ARCH_DF = pd.read_excel(ARCH_path, dtype={'CODE':str}, engine='openpyxl' )
            #print(ARCH_DF)
            #print(Out_EMIRATE_CPI)
            updated_df = pd.merge(ARCH_DF, Out_EMIRATE_CPI, on='CODE', how='left')
            #print(updated_df)
            try:
                arch_col_to_chk=[prc_month+'_'+prc_year+'_UAE']
                for arch_col in arch_col_to_chk:
                    if arch_col not in ARCH_DF.columns:
                        with pd.ExcelWriter(Output_Path, engine='openpyxl') as writer:
                            updated_df.to_excel(writer, sheet_name='Sheet1', index=False)
                        print("ARCH Data updated successfully")
                        Log_DF.loc[len(Log_DF)] = ['ARCH Data updated successfully', current_date]
                    else:
                        print(f"The processing month: {arch_col} already exists in the ARCH output file.")
                        Log_DF.loc[len(Log_DF)] = ['The processing month already exists in the ARCH output file.', current_date]
            except Exception as e:
                print('An error occurred while writing the DataFrame to Excel:'+str(e))
                Log_DF.loc[len(Log_DF)] = ['An error occurred while writing the DataFrame to Excel:'+str(e), current_date]
        except Exception as e:
            print('An error occurred while writing into ARCH process:'+str(e))
            Log_DF.loc[len(Log_DF)] = ['An error occurred while writing into ARCH process: '+str(e), current_date]

ARCH_PRC(arch_flg)

os.makedirs(os.path.dirname(Log_path), exist_ok=True)
Log_DF.to_csv(Log_path, sep='\t', index=False)


source_path = Output_Path
destination_path = '/content/sample_data/Copy_Of_ARCH_TimeSeries.xlsx'

if os.path.exists(destination_path):
    os.remove(destination_path)

shutil.copy(source_path, destination_path)

print(f"File copied to {destination_path}")
print('----------------------------------------------------------------------------------------------------------------------')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rpy2: filename=rpy2-3.5.16-cp310-cp310-linux_x86_64.whl size=329847 sha256=6f12f4d85de68dabc0e686210bb1050f861d3d8280327588c372c11fc6e6bd6a
  Stored in directory: /root/.cache/pip/wheels/86/9b/1a/c09b2935ab01605117b270e2abb744a10f5b213c6f9b7213ce
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.4.2
    Uninstalling rpy2-3.4.2:
      Successfully uninstalled rpy2-3.4.2
Readed
                                          LOGS        TIME
0           

(as ‘lib’ is unspecified)







	‘/tmp/RtmptGPFgB/downloaded_packages’

Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union





Not enough test data for ISIC code: isic0512 after removing NAs

Predictions saved to 'Predictions' sheet in the Excel file.
Predictions Updated to UQ sheet in the Excel file.
UQ Estimation Completed

Not enough test data for ISIC code: isic0512 after removing NAs

Predictions saved to 'Predictions' sheet in the Excel file.
Predictions Updated to AJ sheet in the Excel file.
AJ Estimation Completed

Not enough test data for ISIC code: isic0512 after removing NAs

Predictions saved to 'Predictions' sheet in the Excel file.
Predictions Updated to SH sheet in the Excel file.
SH Estimation Completed
     CODE                           DESC_AR                        DESC_EN  \
0      00                     الإنفاق العام             Public Expenditure   
1      01  الاغذية والمشروبات غير الكحولية   Food And Non-Alcoholic Drinks   
2     011                           الاغذية                          Foods   
3    0111                  الحبوب ومنتجاتها     Cereals And Their Products   
4    01